# Astropy

"The Astropy Project is a community effort to develop a single core package for Astronomy"

Astropy includes many different tools which may be of use to you including (but not limited to):
* units and constants
* time
* tables
* coordinates
* FITS files and WCS
* models and fitting

In addition Astropy includes the concept of affiliated packages:  "An affiliated package is an astronomy-related Python package that is not part of the astropy core package, but has requested to be included as part of the Astropy project’s community."  Some highlights include:

* ginga FITS viewer
* montage image mosaicing engine
* APLpy (Astronomical Plotting Library in Python)

# astropy.io.fits

The `astropy.io.fits` package evolved from the `pyfits` package which you may have encountered in the past.  `pyfits` has been deprected, though it is being maintained so that it keeps up with `astropy.io.fits` for now.

In [1]:
from astropy.io import fits

Astropy provides access to FITS files which requires you to understand a bit more about the FITS file structure than you may have if you've only used FITS viewers in the past.

FITS files are composed of one or more "Header Data Units" (HDUs), so when you open a fits file you will get a list of HDUs.

Each HDU contains a header and data.

In [2]:
hdul = fits.open('km170918_003.fits')
# hdul = fits.open('kb170805_00050_icuber.fits')
print(type(hdul))

<class 'astropy.io.fits.hdu.hdulist.HDUList'>


In [3]:
# This HDU List has only one HDU
hdul

In [4]:
hdul.info()

Filename: km170918_003.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     128   (512, 512)   int16   


In [5]:
hdu = hdul[0]
hdr = hdul[0].header
hdr

 [astropy.io.fits.verify]


SIMPLE  = T                                                                     
BITPIX  = 16                                                                    
NAXIS   = 2                                                                     
NAXIS1  = 512                                                                   
NAXIS2  = 512                                                                   
BZERO   = 0                                                                     
BSCALE  = 1                                                                     
BINNING = 2                                                                     
TARGPMRA= 0 / targ pr motion ra (0 s/yr)                                        
CURRINST= 'KCWI' / current instrument                                           
RAOFF   = 0.000000 / ra offset (0.000000 arcsec)                                
CTYPE2  = 'DEC--TAN' /Second axis type                                          
TVFLIP  = 'no' /porg to tv y

### Headers

In [6]:
hdr.get('AZ')

87.174615

In [7]:
hdr['AZ']

87.174615

In [8]:
hdr['FUBAR']

KeyError: "Keyword 'FUBAR' not found."

In [9]:
result = hdr.get('FUBAR')
print(result)

None


In [10]:
hdr.comments['AZ']

'telescope azimuth (87.174615 deg)'

In [11]:
for key in hdr.keys():
    print(key)

SIMPLE
BITPIX
NAXIS
NAXIS1
NAXIS2
BZERO
BSCALE
BINNING
TARGPMRA
CURRINST
RAOFF
CTYPE2
TVFLIP
CTYPE1
RADESYS
ROTMODE
TARGEQUI
POYPOS9
TARGRA
POXPOS
POYPOS8
POYPOS7
POYPOS6
PONAME9
POYPOS5
PONAME8
TARGEPOC
POYPOS4
PONAME7
FOCALSTN
POYPOS3
PONAME6
POYPOS
POYPOS2
PONAME5
TARGNAME
POYPOS1
PONAME4
PONAME3
PONAME2
PARANTEL
PONAME1
DCSSTAT
TVANGL
DECOFF
DOMESTAT
PARANG
TARGPMDC
POXPOS9
POXPOS8
POXPOS7
TARGFRAM
POXPOS6
POXPOS5
POXPOS4
POXPOS3
POXPOS2
DEC
POXPOS1
CRPIX2
CRPIX1
TELFOCUS
UTC
RA
CUNIT2
CUNIT1
EL
TUBETEMP
DOMEPOSN
AIRMASS
ROTREFAN
SECFOCUS
INSTFLIP
EQUINOX
CELOCAL
TVANDEF
MJD-OBS
TARGRADV
YPXSZ
TELESCOP
INSTANGL
AXESTAT
ROTDEST
CAMPORG
POXOFF
ROTPOSN
CRVAL2
POYOFF
CRVAL1
TARGPLAX
PONAME
LST
SECTHETY
SECTHETX
DATE
DATE-OBS
XPXSZ
CAMREFY
CAMREFX
ROTPDEST
CD2_2
TARGDEC
CD2_1
CALOCAL
PMFM
ROTPPOSN
CD1_2
CD1_1
CAMBIN
HA
AZ
DCSVERS
ROTCALAN
CAMNAME
FRAME
WINDOW
FILTER0
FILTER1
CAMFOCUS
IQLPOS
TTIME
CAMBIAS0
CAMBIAS1
CAMGAIN
TEMP1
TEMP2
TEMP3
TEMP4


In [12]:
hdr.set('AZ', 0.0)

In [13]:
hdr.get('AZ')

0.0

In [14]:
if not hdr.get('FUBAR'):
    hdr.set('FUBAR', True, 'Situation Normal')

In [15]:
hdr['FUBAR']

True

In [16]:
hdr.comments['FUBAR']

'Situation Normal'

### Data

The data in a FITS image file is read out and stored in a numpy array.

In [17]:
im = hdul[0].data
print(type(im))

<class 'numpy.ndarray'>


In [18]:
im.shape

(512, 512)

In [19]:
im

array([[259, 258, 258, ..., 260, 258, 260],
       [259, 260, 260, ..., 261, 261, 260],
       [260, 259, 258, ..., 260, 260, 261],
       ..., 
       [258, 260, 260, ..., 261, 262, 259],
       [260, 259, 259, ..., 260, 259, 260],
       [259, 259, 259, ..., 259, 259, 259]], dtype=int16)

Astropy also supports FITS tables, but we will leave those for you to examine on your own.  As always, information is on the astropy documentation page:
http://docs.astropy.org

### Manipulating FITS Files

In [20]:
hdul = fits.open('km170918_003.fits')
hdul[0].header.get('FUBAR')

In [21]:
if not hdul[0].header.get('FUBAR'):
    hdul[0].header.set('FUBAR', True, 'Situation Normal')
hdul.writeto('newfits.fits', output_verify='ignore')

 [astropy.io.fits.verify]


In [22]:
hdul2 = fits.open('newfits.fits')
hdul2[0].header.get('FUBAR')

True

### Closing FITS Files

FITS files can take up lots of memory if they contain a lot of pixels.  If you're trying to open a single large file, there is an option (`mmap`) which will tell astropy not to read the entire file in to memory.

A more common situation is when you have many files you are opening.  Once you use a file, you may not need it's data after a certain point, so there is no reason for it to remain in memory.  Normally, I would just rely on python's built in memory cleanup to deal with this, but looping through numerous FITS files can overfill your memory.  

The solution is to use the `.close()` method on an `HDUList` when you are done with it, but it is ught to have many `hdul.close()` scattered throughout your code, a cleaner, easier to read solution is to use python's `with`:

In [23]:
with fits.open('newfits.fits', mode='update') as hdul3:
    hdul3[0].header.set('FUBAR', False)
    hdul3.flush(output_verify='ignore')

In [25]:
with fits.open('newfits.fits', mode='readonly') as hdul4:
    fubar = hdul3[0].header.get('FUBAR')
print(fubar)

False


# Astropy Command Line Utilities

For convenience, several of Astropy’s subpackages install utility programs on your system which allow common tasks to be performed without having to open a Python interpreter. These utilities include:

* fitsheader: prints the headers of a FITS file.
* fitscheck: verifies and optionally re-writes the CHECKSUM and DATASUM keywords of a FITS file.
* fitsdiff: compares two FITS files and reports the differences.
* Scripts: converts FITS images to bitmaps, including scaling and stretching.
* wcslint: checks the WCS keywords in a FITS file for compliance against the standards.

# World Coordinate Systems

Astropy includes support for world coordinate system (WCS) representations.  The WCS is a very powerful, generic system for describing the transformation between pixel coordinates and some other system (e.g. RA, Dec).  Because it is so powerful and generic, it is also complicated.

In [26]:
from astropy.wcs import WCS
import numpy as np

In [27]:
w = WCS(hdul[0].header)

In [28]:
w

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 340.43822499999999  12.432842000000001  
CRPIX : 253.0  260.0  
CD1_1 CD1_2  : -0.00010222210000000001  -1.4335179999999999e-07  
CD2_1 CD2_2  : 1.4335179999999999e-07  -0.00010222210000000001  
NAXIS : 512  512

In [29]:
# Three pixel coordinates
pix = np.array([[0, 0],
                [24, 38],
                [45, 98]])
# Conver to world coordinates
world = w.wcs_pix2world(pix, 1)
world

array([[ 340.46474912,   12.45938218],
       [ 340.46223068,   12.45550142],
       [ 340.46002296,   12.44937129]])

In [30]:
# Convert back to pixels
pix2 = w.wcs_world2pix(world, 1)
pix2

array([[  1.47622359e-10,   3.27986527e-11],
       [  2.40000000e+01,   3.80000000e+01],
       [  4.50000000e+01,   9.80000000e+01]])

The `,1` in the conversions above is to specify the origin.  Here, origin is the coordinate in the upper left corner of the image. In FITS and Fortran standards, this is 1. In Numpy and C standards this is 0.

Astropy WCS supports SIP distortions and many projections.  See the astropy documentation for more info on the details.